In [213]:
import re
import pandas as pd
import numpy as np
import MySQLdb
import time
from scipy.ndimage.interpolation import shift
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import train_test_split
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.svm import NuSVC, SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingRegressor
from sklearn.model_selection import cross_val_predict, cross_val_score, StratifiedKFold, KFold
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error
from sklearn import preprocessing

In [214]:
tStart = time.time()
start_timestamp = '1433088000' #1433088000 #1484870400
conn=MySQLdb.connect("54.65.19.253", "news", "newsnews", "news")
sql = "SELECT timestamp, tweet FROM trump WHERE timestamp > %s;"%(start_timestamp)
trump_data = pd.read_sql_query(sql, conn)
sql = "SELECT timestamp, usdx FROM usdx WHERE timestamp > %s;"%(start_timestamp)
usdx_data = pd.read_sql_query(sql, conn)
tEnd = time.time()
conn.close()
print("Fetch data time: " +str(tEnd-tStart)+ "sec.")

Fetch data time: 0.7796633243560791sec.


In [215]:
trump_data.head()

,timestamp,tweet
0,1451491200,I would like to wish everyone A HAPPY AND HEAL...
1,1451491200,"HAPPY BIRTHDAY to my son, @DonaldJTrumpJr! Ver..."
2,1451491200,#MakeAmericaGreatAgain #Trump2016 https://t.co...
3,1451491200,"""@whispers34: Nevada Poll: Donald Trump 33%, T..."
4,1451491200,"""@stylin1188: Donald Trump is running the most..."


In [216]:
trump_data.shape

(6453, 2)

In [217]:
trump_data = trump_data.groupby('timestamp', as_index=False).agg(lambda tweet: ' '.join(tweet))
Y = usdx_data.as_matrix(columns=['usdx']).reshape(-1)
usdx_data['gradient'] = np.gradient(Y)

In [218]:
def get_shift(usdx_data, shift_offset=2):
    g = usdx_data.as_matrix(columns=['gradient']).reshape(-1)
    for i in range(1, shift_offset+1):
        usdx_data['gradient_p_'+str(i)] = shift(g, i, cval=0)

In [219]:
get_shift(usdx_data, shift_offset=2)
usdx_data.head()

,timestamp,usdx,gradient,gradient_p_1,gradient_p_2
0,1433174400,115.3036,-0.29840,0.00000,0.00000
1,1433260800,115.0052,0.03595,-0.29840,0.00000
2,1433347200,115.3755,0.52140,0.03595,-0.29840
3,1433433600,116.0480,0.16395,0.52140,0.03595
4,1433692800,115.7034,-0.44425,0.16395,0.52140


In [220]:
data = pd.merge(trump_data, usdx_data, on='timestamp')

In [221]:
data.head()

,timestamp,tweet,usdx,gradient,gradient_p_1,gradient_p_2
0,1433174400,The Spa @TrumpWaikiki offers unique treatments...,115.3036,-0.29840,0.00000,0.00000
1,1433260800,"""@123Jayne: @realDonaldTrump I have just one q...",115.0052,0.03595,-0.29840,0.00000
2,1433347200,With great patriots in Mason City who also wan...,115.3755,0.52140,0.03595,-0.29840
3,1433433600,We must stop Common Core from controlling stat...,116.0480,0.16395,0.52140,0.03595
4,1433692800,Trump National Golf Club Los Angeles offers 18...,115.7034,-0.44425,0.16395,0.52140


In [222]:
print(data.shape)

(488, 6)


In [223]:
target_names = ['down','up']

In [224]:
def binary(Y):
    Y[np.where(Y > 0)] = 1
    Y[np.where(Y <= 0)] = 0
    Y = Y.astype('int64')
    f = np.bincount(Y)
    print(f)
    return Y

In [225]:
Y = data.as_matrix(columns=['gradient']).reshape(-1)
Y = binary(Y)

[225 263]


In [226]:
X_p = data.as_matrix(columns=['gradient_p_1', 'gradient_p_2'])
X_p[:,0] = binary(X_p[:,0])
X_p[:,1] = binary(X_p[:,1])

[226 262]
[229 259]


In [227]:
vectorizer = TfidfVectorizer(min_df=4, ngram_range=(1, 4))
X = vectorizer.fit_transform(data['tweet'].tolist())
X

<488x4793 sparse matrix of type '<class 'numpy.float64'>'
	with 66784 stored elements in Compressed Sparse Row format>

In [228]:
with_trump = True
if with_trump:
    X = hstack([X, X.power(2), X.power(3), X.power(4), X_p])
else:
    X = X_p

In [229]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8)
print(X_train.shape)
print(X_test.shape)

(390, 19174)
(98, 19174)


In [230]:
def create_model():
    model = NuSVC(kernel='rbf',
              nu=0.5,
              degree = 3,
              verbose=0)
    return model

In [231]:
cv = 5
model = create_model()
recall = cross_val_score(model, X, Y, cv=cv, n_jobs = -1, scoring='recall')
precision = cross_val_score(model, X, Y, cv=cv, n_jobs = -1, scoring='precision')
print('\nprecision:',np.mean(precision))
print('recall:', np.mean(recall))
print('acc: %.4f'%(np.mean(cross_val_score(model, X, Y, cv=cv, n_jobs = -1))))


precision: 0.714362101916
recall: 0.6087808418
acc: 0.6496


In [232]:
def show_result(X_test, y_test):
    print(confusion_matrix(y_test, model.predict(X_test), labels=[0,1]))
    print(classification_report(y_test, model.predict(X_test), target_names=target_names))

In [233]:
skf = StratifiedKFold(n_splits=cv, shuffle=True)
for train_idx, test_idx in skf.split(X, Y):
    print ("Running Fold")
    model = create_model()
    if type(X) != np.ndarray:
        model.fit(X.tocsr()[train_idx], Y[train_idx])
        show_result(X.tocsr()[test_idx], Y[test_idx])
    else:
        model.fit(X[train_idx], Y[train_idx])
        show_result(X[test_idx], Y[test_idx])

Running Fold
[[29 16]
 [12 41]]
             precision    recall  f1-score   support

       down       0.71      0.64      0.67        45
         up       0.72      0.77      0.75        53

avg / total       0.71      0.71      0.71        98

Running Fold
[[41  4]
 [41 12]]
             precision    recall  f1-score   support

       down       0.50      0.91      0.65        45
         up       0.75      0.23      0.35        53

avg / total       0.64      0.54      0.48        98

Running Fold
[[24 21]
 [14 39]]
             precision    recall  f1-score   support

       down       0.63      0.53      0.58        45
         up       0.65      0.74      0.69        53

avg / total       0.64      0.64      0.64        98

Running Fold
[[30 15]
 [15 37]]
             precision    recall  f1-score   support

       down       0.67      0.67      0.67        45
         up       0.71      0.71      0.71        52

avg / total       0.69      0.69      0.69        97

Running Fold